In [6]:
import torch
import dhg
import torch.nn as nn
import torch.nn.functional as F

class GCNConv(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, bias: bool = True, drop_rate: float = 0.5):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, g: dhg.Graph) -> torch.Tensor:
        if not isinstance(g, dhg.Graph):
            raise TypeError(f"Expected dhg.Graph, got {type(g)}")  
        X = self.theta(X)
        X_ = g.smoothing_with_GCN(X)
        X_ = self.drop(self.act(X_))
        return X_

class SSConv(nn.Module):
    '''Spectral-Spatial Convolution'''
    def __init__(self, in_ch, out_ch, kernel_size=3):
        super(SSConv, self).__init__()
        self.depth_conv = nn.Conv2d(
            in_channels=out_ch,
            out_channels=out_ch,
            kernel_size=kernel_size,
            stride=1,
            padding=kernel_size//2,
            groups=out_ch
        )
        self.point_conv = nn.Conv2d(
            in_channels=in_ch,
            out_channels=out_ch,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False
        )
        self.Act1 = nn.LeakyReLU()
        self.Act2 = nn.LeakyReLU()
        self.BN = nn.BatchNorm2d(out_ch)  # Moved after point_conv
        
    def forward(self, input):
        print("Input to point_conv:", input.shape)
        out = self.point_conv(input)
        print("Output of point_conv:", out.shape)  # Apply pointwise conv first
        out = self.BN(out)  # Then batch normalization
        out = self.Act1(out)
        out = self.depth_conv(out)
        out = self.Act2(out)
        return out

class GCNLayer(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, edge_index: torch.Tensor, num_nodes: int):
        super(GCNLayer, self).__init__()
        self.edge_index = edge_index
        self.num_nodes = num_nodes
        self.BN = nn.BatchNorm1d(input_dim)
        self.Activation = nn.LeakyReLU()
        
        # GCN layers
        self.GCN_liner_theta_1 = nn.Linear(input_dim, 256)
        self.GCN_liner_out_1 = nn.Linear(256, output_dim)

    def normalize_adj(self):
        """Computes the normalized adjacency matrix."""
        adj = torch.zeros((self.num_nodes, self.num_nodes), device=self.edge_index.device)
        adj[self.edge_index[0], self.edge_index[1]] = 1  # Convert edge list to adjacency matrix
        D = adj.sum(1)
        D_hat = torch.diag(torch.where(D > 0, torch.pow(D, -0.5), torch.zeros_like(D)))  # Avoid NaNs
        return torch.mm(D_hat, torch.mm(adj, D_hat))

    def forward(self, H):
        H = self.BN(H)
        A_hat = self.normalize_adj()
        H_xx1 = self.GCN_liner_theta_1(H)
        output = torch.mm(A_hat, self.GCN_liner_out_1(H_xx1))
        return self.Activation(output)

class CEGCN(nn.Module):
    def __init__(self, input_dim: int, class_count: int, edge_index: torch.Tensor, num_nodes: int):
        super(CEGCN, self).__init__()
        self.class_count = class_count
        self.num_nodes = num_nodes
        self.edge_index = edge_index
        
        # Spectral-Spatial Convolution
        self.SSConv_layer = SSConv(input_dim, 128, kernel_size=3)

        # GCN layers
        self.GCN_Branch = nn.Sequential(
            GCNLayer(128, 128, self.edge_index, self.num_nodes),
            GCNLayer(128, 64, self.edge_index, self.num_nodes)
        )

        # Softmax classification layer
        self.Softmax_linear = nn.Linear(64, self.class_count)

    def forward(self, x: torch.Tensor):
        """Forward pass using SSConv + GCN for superpixel classification."""
        x = x.unsqueeze(1)  # Shape [B, C, H, W]
        print("Before SSConv:", x.shape)  # Debugging
        x = self.SSConv_layer(x)  # Apply spectral-spatial convolution
        x = x.reshape(self.num_nodes, -1)  # Ensure correct shape
        
        H = self.GCN_Branch(x)  # Pass through GCN layers
        
        Y = self.Softmax_linear(H)  # Classification layer
        return F.log_softmax(Y, dim=-1)


In [7]:
from sklearn.metrics import accuracy_score, classification_report

def train(model, data, optimizer, criterion):
    model.train()
    
    optimizer.zero_grad()
    
    # Pass edge_index instead of graph
    out = model(data['x'])  # Forward pass
    
    # Compute loss
    loss = criterion(out[data['train_mask']], data['y'][data['train_mask']])
    
    loss.backward()
    optimizer.step()

    # Compute accuracy
    pred = out.argmax(dim=1)
    correct = pred[data['train_mask']].eq(data['y'][data['train_mask']]).sum().item()
    acc = correct / data['train_mask'].sum().item()
    
    return loss.item(), acc


def evaluate(model, data):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        x = data['x'].to(device)  # Superpixel features
        y_true = data['y'].cpu().numpy()  # True labels
        output = model(x)
        y_pred = output.argmax(dim=1).cpu().numpy()  # Predicted labels
    
    # Compute classification metrics
    accuracy = accuracy_score(y_true, y_pred)
    print("Accuracy:", accuracy)
    print(classification_report(y_true, y_pred))

    return accuracy


In [8]:
%run data.ipynb

PATH_TO_IMAGE = "E:\MINI_PROJECT\PAVIA_UNI\PaviaU.mat"
PATH_TO_LABEL = "E:\MINI_PROJECT\PAVIA_UNI\PaviaU_gt.mat"

data = DATA(PATH_TO_IMAGE, PATH_TO_LABEL, 1)

Created superpixel graph with 91 nodes and 15936 edges.


In [9]:
graph_data = data.get_data_for_gcn()

# Extract node features
x = graph_data['x']

# Extract edge list from graph
edge_list = torch.stack([torch.tensor(graph_data['graph'].e_src, dtype=torch.long), 
                         torch.tensor(graph_data['graph'].e_dst, dtype=torch.long)], dim=0)

num_nodes = x.shape[0]
graph_data['edge_index'] = edge_list
class_count = len(torch.unique(graph_data['y']))  # Number of classes

# Initialize model
model = CEGCN(input_dim=x.shape[1], class_count=class_count, edge_index=edge_list, num_nodes=num_nodes)


C:\Users\pavan\AppData\Local\Temp\ipykernel_11812\1998630865.py:153: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'x': torch.tensor(self.x, dtype=torch.float32),
C:\Users\pavan\AppData\Local\Temp\ipykernel_11812\1634272797.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_list = torch.stack([torch.tensor(graph_data['graph'].e_src, dtype=torch.long),
C:\Users\pavan\AppData\Local\Temp\ipykernel_11812\1634272797.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(graph_data['graph'].e_dst, dtype=torch.long)], dim=0)


In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, graph_data, optimizer, criterion)
    test_loss, test_acc = evaluate(model, graph_data, criterion)

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

Before SSConv: torch.Size([91, 1, 103])
Input to point_conv: torch.Size([91, 1, 103])


RuntimeError: Given groups=1, weight of size [128, 103, 1, 1], expected input[1, 91, 1, 103] to have 103 channels, but got 91 channels instead